In [1]:
from forcefield import CGForceField
from model_builder import CGModelBuilder

In [2]:
import sys

import openmm
import openmm.app as app
from openmm import unit

builder = CGModelBuilder(
    smiles_sequence="[EO]100",
    bond_length=0.35,  # nm
    min_dist=0.35,  # nm
    max_tries=50000,
)
topology, positions = builder.create_molecule()

with open("peo100.pdb", "w") as f:
    app.PDBFile.writeFile(topology, positions, f)


In [3]:
# structures = [
#     {
#         "pdb": "peo10.pdb",
#         "number": 50,
#         "box": (0, 0, 0, 20, 20, 20),  # Aを10個  50Å立方
#     },
# ]

# topology, positions = builder.create_packed_model(
#     structures,
#     box_size=(20, 20, 20),  # nm
#     output_pdb="peo10_100.pdb",  # 生成されるPACKMOL出力
# )

topology, positions = builder.replicate_model(topology, positions, nx=3, ny=3, nz=3)

In [5]:
param_dict = {
    "BOND": {
        ("EO", "EO"): {"r0": 0.36, "k": 7000},
    },
    "ANGLE": {
        ("EO", "EO", "EO"): {"theta0": 123, "k": 80},
    },
    "TORSION": {
        ("EO", "EO", "EO", "EO"): {"periodicity": 1, "phase": 180.0, "k": 1.96},
    },
    "NONBONDED": {
        "EO": {"sigma": 0.41, "epsilon": 2.35, "charge": 0.0, "mass": 44.0},
    },
}
ff = CGForceField.from_dict(param_dict)
system = ff.create_system(topology)

In [6]:
temperature = 300 * unit.kelvin
friction = 1 / unit.picosecond
timestep = 0.002 * unit.picoseconds
integrator = openmm.LangevinIntegrator(temperature, friction, timestep)

simulation = app.Simulation(topology, system, integrator)
barostat = openmm.MonteCarloBarostat(1 * unit.atmospheres, temperature)
simulation.system.addForce(barostat)

simulation.context.setPositions(positions)
simulation.reporters.append(app.XTCReporter("example-traj-packmol.xtc", 1000))
simulation.reporters.append(
    app.StateDataReporter(
        sys.stdout,
        1000,
        step=True,
        potentialEnergy=True,
        kineticEnergy=True,
        temperature=True,
        volume=True,
        density=True,
    )
)
simulation.reporters.append(
    app.StateDataReporter(
        "md.log",
        1000,
        step=True,
        potentialEnergy=True,
        kineticEnergy=True,
        temperature=True,
        volume=True,
        density=True,
    )
)
simulation.context.reinitialize(preserveState=True)
simulation.minimizeEnergy()
simulation.step(100_000)


#"Step","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)","Density (g/mL)"
1000,17567.243355089504,8950.42521223687,265.79968368136286,1901.0325797197502,0.10377099439786286
2000,18618.374476731304,9989.581527050961,296.6593817653656,1787.130562619355,0.11038479521670536
3000,18710.945304851146,9988.590715197244,296.6299577468345,1685.733828238124,0.11702443047395894
4000,18356.779283112508,10298.610397303832,305.8365743583299,1575.096318466283,0.12524443036749808
5000,17900.666117636443,9870.146410061521,293.1125316924956,1512.3177355661196,0.13044351497101656
6000,17598.819823469497,10446.359297060897,310.22425537777536,1436.7451376244137,0.13730482603646213
7000,17324.97119801479,9846.613442231828,292.41367602286056,1370.0781144751347,0.14398598086929243
8000,16948.549302801188,10009.69695051115,297.25674703752327,1333.1398009495085,0.14797550942500473
9000,16423.8439200247,10369.166607633742,307.9318735137049,1251.6245816734759,0.15761278

KeyboardInterrupt: 

In [8]:
import MDAnalysis as mda
import nglview as nv

bonds = list(topology.bonds())
bonds_nglview = []
for bond in bonds:
    bonds_nglview.append((bond[0].index, bond[1].index))
u = mda.Universe("example-traj-packmol.xtc")
u.add_TopologyAttr("bonds", bonds_nglview)
view = nv.show_mdanalysis(u)
view.add_ball_and_stick()
view.add_representation("ball+stick", colorScheme="ersindex")
view.add_representation("unitcell")
view

/home/mori/work/flowapp/.venv/lib/python3.12/site-packages/MDAnalysis/core/universe.py:1663: UserWarning: there is no reference attributes in this universe to guess types from
  warnings.warn(str(e))
/home/mori/work/flowapp/.venv/lib/python3.12/site-packages/MDAnalysis/core/universe.py:1663: UserWarning: there is no reference attributes (elements, types, or names) in this universe to guess mass from
  warnings.warn(str(e))


NGLWidget(max_frame=63)

In [ ]:
import pandas as pd

df = pd.read_csv("state.log", delim_whitespace=True)